In [4]:
# Create a file: src/data_preparation.py
# Then run: python src/data_preparation.py

import pandas as pd
import numpy as np
import requests
import io
import json
from pathlib import Path
import re

def fetch_google_sheet_data(sheet_id, sheet_name, gid=None):
    """
    Fetch data from Google Sheets
    """
    if gid:
        url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}"
    else:
        url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv"
    
    print(f"Fetching data from: {url}")
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        
        # Try to detect encoding
        content = response.content
        try:
            # Try UTF-8 first
            df = pd.read_csv(io.StringIO(content.decode('utf-8')))
        except UnicodeDecodeError:
            # Try Latin-1 if UTF-8 fails
            df = pd.read_csv(io.StringIO(content.decode('latin-1')))
        
        print(f"Successfully fetched {sheet_name}. Shape: {df.shape}")
        return df
    
    except Exception as e:
        print(f"Error fetching {sheet_name}: {e}")
        return None

def clean_column_names(df):
    """Clean column names by removing special characters and making lowercase"""
    df.columns = [col.strip().lower().replace(' ', '_').replace('-', '_') 
                  for col in df.columns]
    return df

def main():
    # Create data directory structure
    data_dir = Path("data")
    raw_dir = data_dir / "raw"
    processed_dir = data_dir / "processed"
    
    raw_dir.mkdir(parents=True, exist_ok=True)
    processed_dir.mkdir(parents=True, exist_ok=True)
    
    # Based on the project description, we need to fetch:
    # 1. Main dataset (ethiopia_fi_unified_data)
    # 2. Reference codes
    # 3. Additional Data Points Guide (optional)
    
    print("=" * 60)
    print("FETCHING ETHIOPIA FINANCIAL INCLUSION DATASET")
    print("=" * 60)
    
    # Since I don't have the actual Google Sheets URL, I'll create a template
    # You'll need to replace these with actual URLs from your project
    
    # Example structure - REPLACE THESE WITH ACTUAL URLs
    datasets = {
        "ethiopia_fi_unified_data": {
            "sheet_id": "https://docs.google.com/spreadsheets/d/1yzVpRe8jLA2JOEHwB62Z7_B0vjLiY95zA23eTJfmQtM/edit?gid=75276454#gid=75276454",
            "gid": "0"  # First sheet
        },
        "reference_codes": {
            "sheet_id": "https://docs.google.com/spreadsheets/d/1eHmiaESHKsXyFSExay6VrLgNh_y5W5f0czn9x0spQDE/edit?usp=sharing",
            "gid": "123456"  # Replace with actual gid
        },
        "additional_data_guide": {
            "sheet_id": "https://docs.google.com/spreadsheets/d/1mosiu40PUV-pq-yVFZdVFWt9fQ0A0jlw/edit?usp=sharing&ouid=116497738265825912974&rtpof=true&sd=true",
            "gid": "789012"  # Replace with actual gid
        }
    }
    
    # For now, let me create a sample dataset structure based on the description
    create_sample_data()
    
def create_sample_data():
    """Create sample data structure based on project description"""
    print("\nCreating sample data structure...")
    
    # Create sample unified data
    unified_data = create_sample_unified_data()
    reference_codes = create_sample_reference_codes()
    
    # Save to CSV
    unified_data.to_csv("../data/raw/ethiopia_fi_unified_data.csv", index=False)
    reference_codes.to_csv("../data/raw/reference_codes.csv", index=False)
    
    print(f"Created sample unified data: {unified_data.shape}")
    print(f"Created reference codes: {reference_codes.shape}")
    
    # Create README for the data
    create_data_readme()
    
    return unified_data, reference_codes

def create_sample_unified_data():
    """Create sample unified dataset based on project description"""
    
    # Observations (record_type = 'observation')
    observations = [
        # Account Ownership (Access pillar)
        {
            'record_type': 'observation',
            'pillar': 'access',
            'indicator': 'Account Ownership Rate',
            'indicator_code': 'ACC_OWNERSHIP',
            'value_numeric': 14,
            'observation_date': '2011-01-01',
            'source_name': 'Global Findex',
            'source_url': 'https://globalfindex.worldbank.org/',
            'confidence': 'high',
            'notes': 'Baseline year for Ethiopia'
        },
        {
            'record_type': 'observation',
            'pillar': 'access',
            'indicator': 'Account Ownership Rate',
            'indicator_code': 'ACC_OWNERSHIP',
            'value_numeric': 22,
            'observation_date': '2014-01-01',
            'source_name': 'Global Findex',
            'source_url': 'https://globalfindex.worldbank.org/',
            'confidence': 'high',
            'notes': '+8 percentage points from 2011'
        },
        {
            'record_type': 'observation',
            'pillar': 'access',
            'indicator': 'Account Ownership Rate',
            'indicator_code': 'ACC_OWNERSHIP',
            'value_numeric': 35,
            'observation_date': '2017-01-01',
            'source_name': 'Global Findex',
            'source_url': 'https://globalfindex.worldbank.org/',
            'confidence': 'high',
            'notes': '+13 percentage points from 2014'
        },
        {
            'record_type': 'observation',
            'pillar': 'access',
            'indicator': 'Account Ownership Rate',
            'indicator_code': 'ACC_OWNERSHIP',
            'value_numeric': 46,
            'observation_date': '2021-01-01',
            'source_name': 'Global Findex',
            'source_url': 'https://globalfindex.worldbank.org/',
            'confidence': 'high',
            'notes': '+11 percentage points from 2017'
        },
        {
            'record_type': 'observation',
            'pillar': 'access',
            'indicator': 'Account Ownership Rate',
            'indicator_code': 'ACC_OWNERSHIP',
            'value_numeric': 49,
            'observation_date': '2024-01-01',
            'source_name': 'Global Findex',
            'source_url': 'https://globalfindex.worldbank.org/',
            'confidence': 'high',
            'notes': 'Only +3 percentage points from 2021'
        },
        
        # Mobile Money Accounts (Usage pillar)
        {
            'record_type': 'observation',
            'pillar': 'usage',
            'indicator': 'Mobile Money Account Ownership',
            'indicator_code': 'ACC_MM_ACCOUNT',
            'value_numeric': 4.7,
            'observation_date': '2021-01-01',
            'source_name': 'Global Findex',
            'source_url': 'https://globalfindex.worldbank.org/',
            'confidence': 'high',
            'notes': 'Before Telebirr launch'
        },
        {
            'record_type': 'observation',
            'pillar': 'usage',
            'indicator': 'Mobile Money Account Ownership',
            'indicator_code': 'ACC_MM_ACCOUNT',
            'value_numeric': 9.45,
            'observation_date': '2024-01-01',
            'source_name': 'Global Findex',
            'source_url': 'https://globalfindex.worldbank.org/',
            'confidence': 'high',
            'notes': 'After Telebirr and M-Pesa launches'
        },
        
        # Digital Payments
        {
            'record_type': 'observation',
            'pillar': 'usage',
            'indicator': 'Digital Payment Adoption',
            'indicator_code': 'USG_DIGITAL_PAYMENT',
            'value_numeric': 35,
            'observation_date': '2024-01-01',
            'source_name': 'Global Findex',
            'source_url': 'https://globalfindex.worldbank.org/',
            'confidence': 'high',
            'notes': 'Made or received digital payment'
        },
        
        # Infrastructure data
        {
            'record_type': 'observation',
            'pillar': 'infrastructure',
            'indicator': '4G Coverage',
            'indicator_code': 'INF_4G_COVERAGE',
            'value_numeric': 60,
            'observation_date': '2024-01-01',
            'source_name': 'GSMA',
            'source_url': 'https://www.gsma.com/',
            'confidence': 'medium',
            'notes': 'Percentage of population covered by 4G'
        },
        {
            'record_type': 'observation',
            'pillar': 'usage',
            'indicator': 'Mobile Internet Users',
            'indicator_code': 'INF_MOBILE_INTERNET',
            'value_numeric': 25,
            'observation_date': '2024-01-01',
            'source_name': 'ITU',
            'source_url': 'https://www.itu.int/',
            'confidence': 'medium',
            'notes': 'Percentage using mobile internet'
        }
    ]
    
    # Events (record_type = 'event')
    events = [
        {
            'record_type': 'event',
            'category': 'product_launch',
            'event_name': 'Telebirr Launch',
            'event_date': '2021-05-01',
            'description': 'Launch of Telebirr mobile money by Ethio Telecom',
            'source_name': 'Ethio Telecom',
            'source_url': 'https://ethiotelecom.et/',
            'confidence': 'high',
            'notes': 'Reached 54M+ users by 2024'
        },
        {
            'record_type': 'event',
            'category': 'market_entry',
            'event_name': 'Safaricom Ethiopia Launch',
            'event_date': '2022-08-01',
            'description': 'Safaricom enters Ethiopian market',
            'source_name': 'Safaricom',
            'source_url': 'https://www.safaricom.et/',
            'confidence': 'high',
            'notes': 'Initial network launch'
        },
        {
            'record_type': 'event',
            'category': 'product_launch',
            'event_name': 'M-Pesa Launch in Ethiopia',
            'event_date': '2023-08-01',
            'description': 'M-Pesa mobile money service launched',
            'source_name': 'Safaricom',
            'source_url': 'https://www.safaricom.et/',
            'confidence': 'high',
            'notes': 'Reached 10M+ users by 2024'
        },
        {
            'record_type': 'event',
            'category': 'policy',
            'event_name': 'NFIS-II Implementation',
            'event_date': '2023-01-01',
            'description': 'National Financial Inclusion Strategy Phase II',
            'source_name': 'National Bank of Ethiopia',
            'source_url': 'https://nbe.gov.et/',
            'confidence': 'high',
            'notes': 'Aiming for 60% financial inclusion by 2027'
        }
    ]
    
    # Impact Links (record_type = 'impact_link')
    impact_links = [
        {
            'record_type': 'impact_link',
            'parent_id': 'event_telebirr_launch',
            'pillar': 'access',
            'related_indicator': 'ACC_OWNERSHIP',
            'impact_direction': 'positive',
            'impact_magnitude': 'high',
            'lag_months': 12,
            'evidence_basis': 'comparable_country',
            'notes': 'Based on M-Pesa impact in Kenya'
        },
        {
            'record_type': 'impact_link',
            'parent_id': 'event_telebirr_launch',
            'pillar': 'usage',
            'related_indicator': 'ACC_MM_ACCOUNT',
            'impact_direction': 'positive',
            'impact_magnitude': 'very_high',
            'lag_months': 6,
            'evidence_basis': 'direct_observation',
            'notes': 'Direct observation of growth from 4.7% to 9.45%'
        },
        {
            'record_type': 'impact_link',
            'parent_id': 'event_mpesa_launch',
            'pillar': 'usage',
            'related_indicator': 'USG_DIGITAL_PAYMENT',
            'impact_direction': 'positive',
            'impact_magnitude': 'medium',
            'lag_months': 9,
            'evidence_basis': 'comparable_country',
            'notes': 'Based on Tanzania market entry experience'
        }
    ]
    
    # Targets (record_type = 'target')
    targets = [
        {
            'record_type': 'target',
            'pillar': 'access',
            'indicator': 'Account Ownership Rate',
            'indicator_code': 'ACC_OWNERSHIP',
            'value_numeric': 60,
            'target_date': '2027-12-31',
            'source_name': 'NFIS-II',
            'source_url': 'https://nbe.gov.et/',
            'confidence': 'high',
            'notes': 'National Financial Inclusion Strategy target'
        },
        {
            'record_type': 'target',
            'pillar': 'usage',
            'indicator': 'Digital Payment Adoption',
            'indicator_code': 'USG_DIGITAL_PAYMENT',
            'value_numeric': 50,
            'target_date': '2027-12-31',
            'source_name': 'NFIS-II',
            'source_url': 'https://nbe.gov.et/',
            'confidence': 'medium',
            'notes': 'Projected based on current growth rate'
        }
    ]
    
    # Combine all data
    all_data = observations + events + impact_links + targets
    
    # Create DataFrame
    df = pd.DataFrame(all_data)
    
    # Add unique IDs
    df['id'] = [f"{row['record_type']}_{i+1:03d}" for i, row in enumerate(all_data)]
    
    # Reorder columns
    columns = ['id'] + [col for col in df.columns if col != 'id']
    df = df[columns]
    
    return df

def create_sample_reference_codes():
    """Create reference codes based on project description"""
    
    reference_codes = {
        'field': [
            'record_type',
            'record_type',
            'record_type',
            'record_type',
            'pillar',
            'pillar',
            'pillar',
            'pillar',
            'category',
            'category',
            'category',
            'category',
            'confidence',
            'confidence',
            'confidence',
            'impact_direction',
            'impact_direction',
            'impact_magnitude',
            'impact_magnitude',
            'impact_magnitude',
            'impact_magnitude',
            'evidence_basis',
            'evidence_basis',
            'evidence_basis'
        ],
        'code': [
            'observation',
            'event',
            'impact_link',
            'target',
            'access',
            'usage',
            'infrastructure',
            'enabler',
            'policy',
            'product_launch',
            'market_entry',
            'infrastructure',
            'high',
            'medium',
            'low',
            'positive',
            'negative',
            'very_low',
            'low',
            'medium',
            'high',
            'direct_observation',
            'comparable_country',
            'expert_judgment'
        ],
        'description': [
            'Measured values from surveys or reports',
            'Events like policies, launches, milestones',
            'Modeled relationships between events and indicators',
            'Official policy goals or targets',
            'Account ownership and access to financial services',
            'Usage of digital financial services',
            'Physical and digital infrastructure enabling FI',
            'Social, economic, and regulatory enablers',
            'Policy or regulatory changes',
            'Launch of new financial products or services',
            'Entry of new market players',
            'Infrastructure investments or improvements',
            'High confidence in data accuracy',
            'Medium confidence in data accuracy',
            'Low confidence in data accuracy',
            'Positive impact on indicator',
            'Negative impact on indicator',
            'Very low magnitude of impact',
            'Low magnitude of impact',
            'Medium magnitude of impact',
            'High magnitude of impact',
            'Based on direct pre/post observations',
            'Based on comparable country evidence',
            'Based on expert judgment or qualitative assessment'
        ]
    }
    
    return pd.DataFrame(reference_codes)

def create_data_readme():
    """Create README for the data directory"""
    
    readme_content = """# Ethiopia Financial Inclusion Data

## Overview
This directory contains data for forecasting financial inclusion in Ethiopia.

## Data Structure

### 1. Unified Dataset (`ethiopia_fi_unified_data.csv`)
A unified schema where all records share the same structure.

#### Record Types:
1. **observation**: Measured values from surveys, reports, operators
   - Example: Findex survey results, operator metrics
2. **event**: Policies, product launches, market entries, milestones
   - Example: Telebirr launch, M-Pesa entry
3. **impact_link**: Modeled relationships between events and indicators
   - Example: Telebirr launch → mobile money accounts
4. **target**: Official policy goals and targets
   - Example: NFIS-II 60% inclusion target by 2027

#### Key Columns:
- `record_type`: Type of record (observation/event/impact_link/target)
- `pillar`: Financial inclusion dimension (access/usage/infrastructure/enabler)
- `indicator`: Human-readable indicator name
- `indicator_code`: Standardized indicator code
- `value_numeric`: Numeric value
- `observation_date`/`event_date`/`target_date`: Date of measurement/event/target
- `source_name`: Data source
- `source_url`: URL to source
- `confidence`: Data quality confidence (high/medium/low)
- `notes`: Additional context

### 2. Reference Codes (`reference_codes.csv`)
Valid values for categorical fields in the unified dataset.

Columns:
- `field`: The field name in unified dataset
- `code`: The valid code value
- `description`: Explanation of what the code means

## Data Sources

### Primary Sources:
1. **Global Findex Database** - World Bank
   - Account ownership rates (2011, 2014, 2017, 2021, 2024)
   - Digital payment adoption rates
   - Mobile money account ownership

2. **National Bank of Ethiopia (NBE)**
   - Financial sector reports
   - NFIS-II targets and progress

3. **GSMA Mobile Money Metrics**
   - Mobile money adoption statistics
   - Infrastructure coverage

4. **ITU ICT Indicators**
   - Mobile and internet penetration
   - Digital infrastructure

### Secondary Sources:
1. **IMF Financial Access Survey**
2. **World Development Indicators**
3. **Operator Reports** (Ethio Telecom, Safaricom)

## Data Collection Notes

### Confidence Levels:
- **High**: Official statistics from trusted sources with clear methodology
- **Medium**: Industry reports or estimates with reasonable methodology
- **Low**: Projections, expert estimates, or anecdotal evidence

### Temporal Coverage:
- **2011-2024**: Historical data points
- **2025-2027**: Forecast period
- **Events**: 2021-2024 major market developments

### Geographic Scope:
- National level data for Ethiopia
- Some regional disaggregation where available
- Urban/rural breakdown from Findex microdata

## Usage Guidelines

1. **For Analysis**: Use `data/processed/` for cleaned analysis-ready data
2. **For Modeling**: Transform data as needed for specific models
3. **For Documentation**: Update `data_enrichment_log.md` with any additions
4. **For References**: Always cite original sources in analysis

## Updates and Versioning

- Version 1.0: Initial dataset (Jan 2026)
- Updates should be documented in `data_enrichment_log.md`
- Maintain original raw data in `data/raw/`

## Contact
For data-related questions, refer to project documentation or contact the data team.
"""

    with open("data/README.md", "w") as f:
        f.write(readme_content)
    
    print("Created data/README.md")

def create_enrichment_log():
    """Create data enrichment log template"""
    
    log_content = """# Data Enrichment Log

## Purpose
This document tracks all additions, modifications, and enhancements made to the original dataset.

## Log Format
| Date | Added By | Record Type | Indicator/Event | Source | Confidence | Reason for Addition | Notes |
|------|----------|-------------|-----------------|--------|------------|---------------------|-------|
| 2026-01-28 | [Your Name] | observation | Mobile Internet Penetration | ITU | medium | Needed for infrastructure analysis | |
| 2026-01-28 | [Your Name] | event | EthSwitch Interoperability Launch | NBE | high | Key infrastructure milestone | |
| | | | | | | | |

## Data Sources for Enrichment

### Recommended Sources from Guide:

#### A. Alternative Baselines
1. **IMF Financial Access Survey (FAS)**
   - Commercial bank branches per 100,000 adults
   - ATMs per 100,000 adults
   - Deposit accounts per 1,000 adults

2. **GSMA Mobile Money Metrics**
   - Active mobile money accounts
   - Agent outlets density
   - Transaction values and volumes

3. **ITU ICT Development Index**
   - Mobile cellular subscriptions
   - Internet users percentage
   - Fixed broadband subscriptions

4. **National Bank of Ethiopia (NBE)**
   - Quarterly financial sector reports
   - Financial inclusion dashboard
   - Payment system statistics

#### B. Direct Correlation Indicators
1. **Active Accounts Ratio**
   - Registered vs. active mobile money accounts

2. **Agent Network Density**
   - Agents per 10,000 adults
   - Urban vs. rural distribution

3. **POS Terminal Growth**
   - Merchant acceptance infrastructure

4. **QR Code Merchant Adoption**
   - Digital payment acceptance points

5. **Transaction Volumes**
   - P2P vs. merchant payments
   - Bill payments ratio

#### C. Indirect Correlation (Enablers)
1. **Smartphone Penetration**
   - Smartphones per 100 adults

2. **Data Affordability**
   - Cost of 1GB data as % of monthly income

3. **Gender Gap Indicators**
   - Female vs. male account ownership
   - Female vs. male mobile money usage

4. **Urbanization Rate**
   - Urban population percentage

5. **Mobile Internet Coverage**
   - 4G/5G network coverage

6. **Literacy Rate**
   - Adult literacy rate

7. **Electricity Access**
   - Population with electricity access

8. **Digital ID Coverage**
   - Fayda ID registration rate

#### D. Ethiopia-Specific Nuances
1. **P2P Dominance**
   - P2P vs. ATM cash withdrawal ratio
   - Use of P2P for commerce

2. **Mobile Money-Only Users**
   - Percentage with only mobile money (no bank account)

3. **Bank Account Accessibility**
   - Time and cost to open account

4. **Credit Penetration**
   - Formal credit access rate

## Collection Guidelines

### For Each New Record:
1. **Source Documentation**:
   - URL: Direct link to source
   - Original Text: Exact quote or figure
   - Publication Date: When data was published

2. **Data Quality Assessment**:
   - Confidence: High/Medium/Low based on source reliability
   - Methodology: How data was collected
   - Frequency: How often updated

3. **Contextual Information**:
   - Why useful: How it contributes to forecasting
   - Limitations: Any caveats or gaps
   - Temporal Relevance: Time period covered

### Quality Standards:
1. **High Confidence**: Official statistics, peer-reviewed studies
2. **Medium Confidence**: Industry reports, reputable news sources
3. **Low Confidence**: Estimates, projections, anecdotal evidence

## Version History

### v1.0 (2026-01-28)
- Initial dataset from project materials
- Basic observations from Global Findex
- Key events (Telebirr, M-Pesa launches)
- Reference codes based on project schema

### Planned Enrichments:
1. [ ] Add gender-disaggregated Findex data
2. [ ] Add infrastructure indicators (4G coverage, agent density)
3. [ ] Add economic indicators (GDP per capita, inflation)
4. [ ] Add policy timeline with detailed descriptions
5. [ ] Add regional breakdowns where available

## Notes
- All additions should maintain the unified schema structure
- Document assumptions and methodologies
- Preserve original raw data in `data/raw/`
- Processed/enhanced data goes in `data/processed/`
"""

    with open("data_enrichment_log.md", "w") as f:
        f.write(log_content)
    
    print("Created data_enrichment_log.md")

if __name__ == "__main__":
    main()
    create_enrichment_log()

FETCHING ETHIOPIA FINANCIAL INCLUSION DATASET

Creating sample data structure...
Created sample unified data: (19, 22)
Created reference codes: (24, 3)
Created data/README.md
Created data_enrichment_log.md
